<a href="https://colab.research.google.com/github/antoniomuso/speech2face/blob/master/Speech2Face_newDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip3 install face_recognition
#! pip install --upgrade wandb
#! wandb login f9cd4b35bf9733e5ead9d2b06e13ef2259b1284e

In [ ]:
#import wandb
#wandb.init(project="speech2face")


In [ ]:
# path = "/content/drive/My Drive/Speech2Face/vox"
# !curl --user voxceleb1912:0s42xuw6 -o "/content/drive/My Drive/Speech2Face/ff/vox.zip" http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip



! cp "/content/drive/My Drive/Speech2Face/vox1_dataset/wav_filtered_20_per_actor.zip" .
#! cp "/content/drive/My Drive/Speech2Face/zippedFaces.tar.gz" /content
! cp "/content/drive/My Drive/Speech2Face/vox1_dataset/vox1_meta.csv" .
! cp "/content/drive/My Drive/Speech2Face/face_features_10_per_actor.zip" .
! cp "/content/drive/My Drive/Speech2Face/vox1_dataset/vox_audios/vox.zip" .

# ! tar zxvf zippedFaces.tar.gz
! unzip face_features_10_per_actor.zip
! unzip wav_filtered_20_per_actor.zip
! unzip vox.zip



In [ ]:
import librosa
import numpy as np
import pandas as pd
from os import listdir
from os.path import join
from torch.utils.data import Dataset
import glob
import itertools
from PIL import Image
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader

import torch.nn.functional as F
import torch.nn as nn
from typing import Optional, Callable
from tqdm.notebook import tqdm, trange

from google.colab.patches import cv2_imshow
import cv2

device = 'cuda'
vgg_weights_path = '/content/drive/My Drive/Speech2Face/Pretrained/vgg_face_dag.pth'
face_decoder_weights_path = ''

In [ ]:
# Reproducibility stuff

import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)

In [ ]:
##################### DEPENDECIES ###########################
class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0, is_fc8=False):
      if not is_fc8:
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        x38 = x37
      else:
        x38 = self.fc8(x0)

      return x38


def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [ ]:
########################### DEPENDENCY - standard DECODER (w/o warping) ##########################
class Decoder(nn.Module):
    def __init__(self, n_hidden, bottom_width=4, channels=512):
        super().__init__()
        self.channels = channels
        self.bottom_width = bottom_width

        self.linear = nn.Linear(n_hidden, bottom_width*bottom_width*channels)
        self.dconv1 = nn.ConvTranspose2d(channels, channels // 2, 4, 2, 1)
        self.dconv2 = nn.ConvTranspose2d(channels // 2, channels // 4, 4, 2, 1)
        self.dconv3 = nn.ConvTranspose2d(channels // 4, channels // 8, 4, 2, 1)
        self.dconv4 = nn.ConvTranspose2d(channels // 8, 3, 4, 2, 1)

        self.bn0 = nn.BatchNorm1d(bottom_width*bottom_width*channels)
        self.bn1 = nn.BatchNorm2d(channels // 2)
        self.bn2 = nn.BatchNorm2d(channels // 4)
        self.bn3 = nn.BatchNorm2d(channels // 8)

    def forward(self, x):
        x = F.relu(self.bn0(self.linear(x))).view(-1, self.channels, self.bottom_width, self.bottom_width)
        x = F.relu(self.bn1(self.dconv1(x)))
        x = F.relu(self.bn2(self.dconv2(x)))
        x = F.relu(self.bn3(self.dconv3(x)))

        x = torch.sigmoid(self.dconv4(x))

        return x

def decoder(weights_path="/content/drive/My Drive/Speech2Face/models/face_decoder/epoch_3_steps_12800.pth", fc3_only=False, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Decoder(4096)
    if weights_path:
        state_dict = torch.load(weights_path)["model_state_dict"]
        model.load_state_dict(state_dict)
        
        if fc3_only:
          fc3_layer = nn.Sequential(list(model.children())[0])
          for param in fc3_layer.parameters():
            param.requires_grad = False
          #print(fc3_layer)
          return fc3_layer        

    return model

# dec_fc3 = decoder(fc3_only=True)

In [ ]:
########################### DEPENDENCY - DECODER w/ warping ##########################

class DECODER(nn.Module):
    def __init__(self, phase):
        super(DECODER, self).__init__()
        self.phase = phase
        self.fc3 = nn.Linear(4096, 1000)
        self.ReLU = nn.ReLU()
        #self.fc_bn3 = nn.BatchNorm1d(1000)


        self.fc4 = nn.Linear(1000, 14 * 14 * 256)
        self.fc_bn4 = nn.BatchNorm1d(14 * 14 * 256)
        def TransConv( i, kernal = 5, stride = 2, inp = None):
            if not inp:
                inp = max(256//2**(i-1), 32)

            layer =  nn.Sequential(
                nn.ConvTranspose2d(inp, max(256//2**i, 32), 
                                kernal, stride=stride, padding=2, output_padding=1, 
                                dilation=1, padding_mode='zeros'),
                nn.ReLU(),
                nn.BatchNorm2d(max(256//2**i, 32)))
            return layer
        self.T1_ = TransConv(1, inp = 256)
        self.T2_ = TransConv(2)
        self.T3_ = TransConv(3)
        self.T4_ = TransConv(4)
    
        self.ConvLast = nn.Sequential(
            nn.Conv2d(32, 3, (1,1), stride=1),
            nn.BatchNorm2d(3),
            nn.ReLU())


        self.layerLandmark1 = nn.Linear(1000, 800)
        self.layerLandmark2 = nn.Linear(800, 600)
        self.layerLandmark3 = nn.Linear(600, 400)
        self.layerLandmark4 = nn.Linear(400, 200)
        self.layerLandmark5 = nn.Linear(200, 144)

    def forward(self, x):
        L1 = self.fc3(x)
        L1 = self.ReLU(L1)


        L2 = self.layerLandmark1(L1)
        L2 = self.ReLU(L2)

        L3 = self.layerLandmark2(L2)
        L3 = self.ReLU(L3)

        L4 = self.layerLandmark3(L3)
        L4 = self.ReLU(L4)

        L5 = self.layerLandmark4(L4)
        L5 = self.ReLU(L5)

        L6 = self.layerLandmark5(L5)
        outL = self.ReLU(L6)


        # B1 = self.fc_bn3(L1) 
        T0 = self.fc4(L1) 
        T0 = self.ReLU(T0)
        # T0 = self.fc_bn4(T0)
        T0 = T0.view(-1,256,14,14)



        T1 = self.T1_(T0)
        T2 = self.T2_(T1)
        T3 = self.T3_(T2)
        T4 = self.T4_(T3)

        outT = self.ConvLast(T4)
        if self.phase == "train":
            return outL,  outT
        elif self.phase == "eval":
            return outT

def decoder_warping(weights_path="/content/drive/My Drive/Speech2Face/models/face_decoder_warping/new_dataset_v4/epoch_94_steps_0.pth", fc3_only=False, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = DECODER('eval')
    if weights_path:
        state_dict = torch.load(weights_path)["model_state_dict"]
        model.load_state_dict(state_dict)

        if fc3_only:
          fc3_layer = nn.Sequential(list(model.children())[0])
          for param in fc3_layer.parameters():
            param.requires_grad = False
          #print(fc3_layer)
          return fc3_layer

    return model

# dec_fc3 = decoder_warping(fc3_only=True)

In [ ]:
def load_wav(wav_path):
  def adjust(stft):
    if stft.shape[1] == 601:
      return stft
    else:
      return np.concatenate((stft,stft[:,0:601 - stft.shape[1]]),axis = 1)

  wav, sr = librosa.load(wav_path,sr = 16000, duration = 6.0 ,mono = True)
  spectro = librosa.core.stft(wav, n_fft = 512, hop_length = int(np.ceil(0.01 * sr)),win_length = int(np.ceil(0.025 * sr)) , window='hann', center=True,pad_mode='reflect') 
  spectroComplex = adjust(spectro)
  converted = np.zeros((spectroComplex.shape[0], spectroComplex.shape[1], 2))
  i = np.arange(spectroComplex.shape[0])
  j = np.arange(spectroComplex.shape[1])
  
  converted[i,j[:,np.newaxis], 0] = spectroComplex[i,j[:,np.newaxis]].real
  converted[i,j[:,np.newaxis], 1] = spectroComplex[i,j[:,np.newaxis]].imag

  return converted

In [ ]:
def get_map_person2paths(path, format='wav'):
  actor2data = dict()
  
  for person in listdir(path):
    n_path = join(path, person)
    files = glob.glob(n_path + '/**/*.'+format, recursive=True)
    actor2data[person] = files
  
  return actor2data

def get_map_person2paths_new_dataset(path):
  actor2data = dict()
  images_path = join(path, 'Faces')
  for image in listdir(images_path):
    person = '_'.join(image.split('_')[:-1])
    # idx = int(image.split('_')[-1].split('.')[0])
    if not person in actor2data.keys():
      actor2data[person] = []
    actor2data[person] += [join(images_path, image)]
  
  return actor2data


def load_metadata(path):
  meta = pd.read_csv(path,sep='\t')

  meta = meta.drop('Gender',axis=1)
  meta = meta.drop('Nationality',axis=1)
  meta = meta.drop('Set',axis=1)
  return meta

def couple_data(voice_map, face_map, meta):
  count = 0
  out = []
  for index, row in meta.iterrows():
    if (row['VoxCeleb1 ID'] not in voice_map.keys()) or (row['VGGFace1 ID'] not in face_map.keys()):
      count += 1
      continue
    # max(len(voice_map[row['VoxCeleb1 ID']]), face_map[row['VGGFace1 ID']])
    coupled = list(zip(voice_map[row['VoxCeleb1 ID']], face_map[row['VGGFace1 ID']]))
    out += coupled
  
  print("elements not found:", count)
  return out

def create_coupled_list(path_voices, path_faces, metaP):
  voice_map = get_map_person2paths(path_voices)
  face_map = get_map_person2paths_new_dataset(path_faces)
  meta = load_metadata(metaP)
  return couple_data(voice_map, face_map, meta)

In [ ]:
class _Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):
        # Convert image to Tensor
        img = transforms.functional.to_tensor(img)

        # Normalize image by the parameter of pre-trained face-encoder
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

class Speech2FaceDataset(Dataset):
  def __init__(self, path_voices, path_faces, metaP, size=224):
        super().__init__()
        self.path_voices = path_voices
        self.path_faces = path_faces
        self.size = size
        self.coupled_list = create_coupled_list(path_voices, path_faces, metaP)
        self.len = len(self.coupled_list)
        self.features = np.load(join(path_faces,'facefeature.npy'))
        self.features_fc8 = np.load(join(path_faces,'facefeature_fc8.npy'))

        self.transform_fd = transforms.Compose([
            transforms.Resize((self.size, self.size)),
            transforms.ToTensor()
        ])
  
  def __len__(self):
        return self.len
    
  def __getitem__(self, idx):
        wav_p, img_p = self.coupled_list[idx]
        wav = load_wav(wav_p)
        img = Image.open(img_p)
        idx_feature = int(img_p.split('_')[-1].split('.')[0])

        
        #face_loc = face_locations(np_image)
        # img_normalized = self.transform_fe(img)
        image_preprocessed = self.transform_fd(img)
        
        return (torch.tensor(self.features[idx_feature]).float(),
                torch.tensor(wav).reshape(2,257,601).float()), (torch.tensor(self.features_fc8[idx_feature]).float() , image_preprocessed)



color_mean = [129.186279296875, 104.76238250732422, 93.59396362304688]
color_std = [1, 1, 1]   
color_mean = [tmp / 255.0 for tmp in color_mean]
color_std = [tmp / 255.0 for tmp in color_std]


data = Speech2FaceDataset('wav_filtered_20_per_actor', 'Face_Feature','vox1_meta.csv')
data_test = Speech2FaceDataset('wav', 'Face_Feature','vox1_meta.csv')



In [ ]:
#data[1]

In [ ]:
class SpeechEncoder(nn.Module):

    def __init__(self):
        super(SpeechEncoder, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, kernel_size=4,stride=1) 
        self.conv2 = nn.Conv2d(64, 64, kernel_size=4,stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4,stride=1) 
        self.pooling1 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4,stride=1) 
        self.pooling2 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv5 = nn.Conv2d(128, 128, kernel_size=4,stride=1) 
        self.pooling3 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv6 = nn.Conv2d(128, 256, kernel_size=4,stride=1) 
        self.pooling4 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv7 = nn.Conv2d(256, 512, kernel_size=4,stride=1) 
        self.conv8 = nn.Conv2d(512, 512, kernel_size=4,stride=2) 

        self.conv9 = nn.Conv2d(512, 512, kernel_size=3,stride=2) # Queste due celle sono diverse
        self.pooling5 = nn.AvgPool2d(kernel_size=(1,1), stride=1)# Queste due celle sono diverse

        self.fc1 = nn.Linear(512 * 1 * 144, 4096)
        self.fc2 = nn.Linear(4096, 4096)

        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.batch_norm5 = nn.BatchNorm2d(128)
        self.batch_norm6 = nn.BatchNorm2d(256)
        self.batch_norm7 = nn.BatchNorm2d(512)
        self.batch_norm8 = nn.BatchNorm2d(512)
        self.batch_norm9 = nn.BatchNorm2d(512)
      


        self.relu = nn.ReLU()
        

    def forward(self, x):
        out = self.batch_norm1(self.relu(self.conv1(x)))
        out = self.batch_norm2(self.relu(self.conv2(out)))
        out = self.batch_norm3(self.relu(self.conv3(out)))
        out = self.pooling1(out)
        out = self.batch_norm4(self.relu(self.conv4(out)))
        out = self.pooling2(out)
        out = self.batch_norm5(self.relu(self.conv5(out)))
        out = self.pooling3(out)
        out = self.batch_norm6(self.relu(self.conv6(out)))
        out = self.pooling4(out)
        out = self.batch_norm7(self.relu(self.conv7(out)))
        out = self.batch_norm8(self.relu(self.conv8(out)))
        out = self.batch_norm9(self.relu(self.pooling5(self.conv9(out))))

        batch = out.shape[0]
        out = out.view((batch, 512 * 1 * 144))
        out = self.relu(self.fc1(out))
        out = self.fc2(out)

        return out

def speech_encoder(weights_path="/content/drive/My Drive/Speech2Face/models/speech_encoder/adam/adam_epoch_9.pth", **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = SpeechEncoder()
    if weights_path:
        state_dict = torch.load(weights_path)["model_state_dict"]
        model.load_state_dict(state_dict)
    return model

In [ ]:
vgg = vgg_face_dag(vgg_weights_path)
vgg.eval()
vgg_fc8 = vgg.fc8
vgg_fc8.requires_grad = False
vgg_fc8.to(device)

# vgg = vgg.to(device)


model = SpeechEncoder()
model.to(device)

decoder = decoder_warping(fc3_only=True)
decoder.to(device)




optimizer = torch.optim.Adam(model.parameters(), eps=1e-04, betas=(0.5, 0.999))    #, weight_decay=0.95)
#optimizer_decay = torch.optim.AdamW(model.parameters(), eps=1e-04, betas=(0.5, 0.999))    #, weight_decay=0.95)
#optimizer = optimizer_decay

datal = DataLoader(data, 6, True, num_workers=16)
datal_test = DataLoader(data_test, 6, False, num_workers=16)
# W&B
#wandb.watch(model)


In [ ]:
def _save_model(epoch, model, optimizer, output_dir_name="/content/drive/My Drive/Speech2Face/models/speech_encoder/"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, join(output_dir_name, 'adam_epoch_{}.pth'.format(epoch)))
    
def make_averager() -> Callable[[Optional[float]], float]:
    """ Returns a function that maintains a running average

    :returns: running average function
    """
    count = 0
    total = 0

    def averager(new_value: Optional[float]) -> float:
        """ Running averager

        :param new_value: number to add to the running average,
                          if None returns the current average
        :returns: the current average
        """
        nonlocal count, total
        if new_value is None:
            return total / count if count else float("nan")
        count += 1
        total += new_value
        return total / count

    return averager

def fit(
    epochs: int,
    train_dl: torch.utils.data.DataLoader,
    test_dl: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    VGG: torch.nn.Module,
    decoder: torch.nn.Module,
    opt: torch.optim.Optimizer,
    tag: str,
    device: str = "cuda",
    restart_epoch: int = 0,
) -> float:
    """ Train the model and computes metrics on the test_loader at each epoch

    :param epochs: number of epochs
    :param train_dl: the train dataloader
    :param test_dl: the test dataloader
    :param model: the model to train
    :param opt: the optimizer to use to train the model
    :param tag: description of the current model
    :param perm: if not None, permute the pixel in each image according to perm

    :returns: accucary on the test set in the last epoch
    """

    print("VGG.training = ", VGG.training)
    print("Speech2Face.training = ", model.training)

    print("Starting training. Restart epoch:", restart_epoch)

    lambda1 = 0.025
    lambda2 = 200.0

    for epoch in trange(epochs, desc="train epoch"):
        if restart_epoch != 0:
          epoch = restart_epoch
          restart_epoch = 0
          print("Epoch updated, current value:", epoch)

        model.train()
        train_loss_averager = make_averager()  # mantain a running average of the loss

        # TRAIN
        tqdm_iterator = tqdm(
            enumerate(train_dl),
            total=len(train_dl),
            desc=f"batch [loss: None]",
            leave=False,
        )
        for batch_idx, (x, y) in tqdm_iterator:
            embedding, wav = x
            features_fc8, img = y
            # send to device
            wav = wav.to(device)
            embedding = embedding.to(device)
            features_fc8 = features_fc8.to(device)
            # print(wav.shape)

            # embedding = VGG(img_normal)# .to(device)

            output = model(wav)

            # print(output.shape)

            fvgg_s = VGG(output)
            fvgg_f = features_fc8

            fdec_s = decoder(output)
            fdec_f = decoder(embedding)
            

            # loss = F.l1_loss(output, embedding)
            loss = F.l1_loss(fdec_f, fdec_s) + lambda1 * loss_2(output, embedding) + lambda2 * dist_loss(fvgg_f, fvgg_s)
            loss.backward()
            opt.step()
            opt.zero_grad()
            #print('here')

            train_loss_averager(loss.item())

            tqdm_iterator.set_description(
                f"train batch [avg loss: {train_loss_averager(None):.3f}]"
            )
            tqdm_iterator.refresh()

         # TEST
         model.eval()
         test_loss_averager = make_averager()  # mantain a running average of the loss
         correct = 0
         for (x, y) in test_dl:
             # send to device
             embedding, wav = x
             features_fc8, img = y
             # send to device
             wav = wav.to(device)
             embedding = embedding.to(device)
             features_fc8 = features_fc8.to(device)
             # print(wav.shape)
 
             # embedding = VGG(img_normal)# .to(device)
 
             output = model(wav)
 
             # print(output.shape)
 
             fvgg_s = VGG(output)
             fvgg_f = features_fc8
 
             fdec_s = decoder(output)
             fdec_f = decoder(embedding)
             

             # loss = F.l1_loss(output, embedding)
             loss = F.l1_loss(fdec_f, fdec_s) + lambda1 * loss_2(output, embedding) + lambda2 * dist_loss(fvgg_f, fvgg_s)
             test_loss_averager(loss)

        print(
            f"Epoch: {epoch}\n"
            f"Train set: Average loss: {train_loss_averager(None):.4f}\n"
            f"Test set: Average loss: {test_loss_averager(None):.4f}, "
            #f"Accuracy: {correct}/{len(test_dl.dataset)} ({accuracy:.0f}%)\n"
        )
        _save_model(epoch, model, optimizer)
        #torch.save(model.state_dict(), join(wandb.run.dir, 'model.pt'))
        #wandb.log({"Train set Average loss:": train_loss_averager(None)})
    # models_accuracy[tag] = accuracy
    # return accuracy

In [ ]:
# taken from https://discuss.pytorch.org/t/is-this-loss-function-for-knowledge-distillation-correct/69863

def dist_loss(t, s):
  T = 2
  prob_t = F.softmax(t/T, dim=1)
  log_prob_s = F.log_softmax(s/T, dim=1)
  dist_loss = -(prob_t*log_prob_s).sum(dim=1).mean()
  return dist_loss

def loss_2(vf, vs):
  return F.mse_loss(F.normalize(vf), F.normalize(vs))
  


In [ ]:
check_path = None #"/content/drive/My Drive/Speech2Face/models/speech_encoder/adam_epoch_2.pth"

def _load_checkpoint(checkpoint_path):
  checkpoint = torch.load(checkpoint_path)
  global_ep = checkpoint["epoch"]
  model.load_state_dict(checkpoint["model_state_dict"])
  optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
  print("Loaded checkpoint, restart epoch is: ", global_ep)
  return global_ep + 1

global_ep = 0
if check_path is not None:
  global_ep = _load_checkpoint(check_path)  

fit(10, datal, datal_test, model, vgg_fc8, decoder, optimizer, "Speech2Face Training", restart_epoch=global_ep)

In [ ]:
from torchsummary import summary


model = SpeechEncoder()
input = torch.unsqueeze(torch.tensor(converted).reshape(2,257,601), 0)

model(input.type(torch.float32)).shape
summary(model, (2,257,601))

In [ ]:
# Quick testing -- models creation

enc = speech_encoder()
dec = decoder()
dec_w = decoder_warping()

In [ ]:
# Quick testing -- actual test

enc.eval()
dec.eval()
#dec_w.eval() # Is it needed??

test_wav_path = "/content/drive/My Drive/Speech2Face/vox1_dataset/vox_audios/ext/wav/id10279/3qAxPgeIvCQ/00001.wav"
test_wav = load_wav(test_wav_path)
test_wav = torch.tensor(test_wav).reshape(2,257,601).float().unsqueeze(0)

test_wav_path2 = "/content/drive/My Drive/Speech2Face/vox1_dataset/vox_audios/ext/wav/id10277/0rpfN7wThsg/00001.wav"
test_wav2 = load_wav(test_wav_path2)
test_wav2 = torch.tensor(test_wav2).reshape(2,257,601).float().unsqueeze(0)

print(torch.equal(test_wav, test_wav2))

#print(test_wav, test_wav2)


out = enc(test_wav)
decoded = dec(out)
decoded_w = dec_w(out)


out2 = enc(test_wav2)
decoded2 = dec(out2)
decoded_w2 = dec_w(out2)

In [ ]:
# Quick testing -- showing output

#1 - Decoder w/ warping
o_img_w = cv2.cvtColor(np.einsum('abc->bca',decoded_w[0].detach().numpy()*255), cv2.COLOR_BGR2RGB)
cv2_imshow(o_img_w)

o_img_w2 = cv2.cvtColor(np.einsum('abc->bca',decoded_w2[0].detach().numpy()*255), cv2.COLOR_BGR2RGB)
cv2_imshow(o_img_w2)

#2 - Decoder w/o warping
o_img = cv2.cvtColor(np.einsum('abc->bca',decoded[0].detach().numpy()*255),cv2.COLOR_BGR2RGB)
cv2_imshow(o_img)

o_img2 = cv2.cvtColor(np.einsum('abc->bca',decoded2[0].detach().numpy()*255),cv2.COLOR_BGR2RGB)
cv2_imshow(o_img2)